In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from coffea import util
from coffea.processor import accumulate
import os, sys
import re
import pandas as pd
import hist
import matplotlib as mpl
import mplhep
import utils
from pathlib import Path

sys.path.append("../../../analysisTools/")
import plotTools as ptools

In [2]:
def add_signal_info_to_df(df):
    m1_list = []
    delta_list = []
    ctau_list = []
    
    for point in df.index.values:
        sig_dict = ptools.signalPoint(point)
        m1_list.append(sig_dict['m1'])
        delta_list.append(sig_dict['delta'])
        ctau_list.append(sig_dict['ctau'])
    
    df['m1'] = m1_list
    df['delta'] = delta_list
    df['ctau'] = ctau_list
    
    df = df.sort_values(by=['m1']) # sort by m1

    return df

In [3]:
def save_df_to_csv(df, outdir, outname, isSignal = False):
    Path(outdir).mkdir(parents=True, exist_ok=True)

    if isSignal:
        df = add_signal_info_to_df(df)
    
    df.to_csv(f'{outdir}/{outname}.csv')

    print(f'Saved: {outdir}/{outname}.csv')

# SR ctau-BDT-based (trained on [5,100]GeV): run on all signal mass points [5, 100] GeV

In [4]:
ct = 10

## Background

In [5]:
prefix = './coffea/skimmed/'
suffix = f'_SR_BDT_ctau-{ct}'

In [6]:
# Background
bkg_files = {
    "QCD": "bkg_2018_QCD_TuneCP5_PSWeights", # No entry for QCD
    "Top": "bkg_2018_Top",
    "DY": "bkg_2018_DY",
    "Diboson": "bkg_2018_Diboson",
    "Triboson": "bkg_2018_Triboson",
    "W+jets": "bkg_2018_WJets",
    "Z+jets": "bkg_2018_ZJets",
}

bkg_histos = {}

for b in bkg_files.keys():
    bkg_histos[b] = util.load(f'{prefix}/{bkg_files[b]}{suffix}.coffea')[0]

In [7]:
# efficiency
pd.options.display.float_format = '{:,.6f}'.format
bkg_eff_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow')

bkg_eff_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
QCD,0.000002,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Top,0.000129,0.000013,0.000013,0.000008,0.000007,0.000007,0.000007,0.000007,0.000000
DY,0.000011,0.000001,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Diboson,0.000637,0.000044,0.000044,0.000033,0.000031,0.000030,0.000030,0.000027,0.000000
Triboson,0.001933,0.000158,0.000158,0.000104,0.000092,0.000091,0.000091,0.000076,0.000000
W+jets,0.002310,0.000173,0.000173,0.000139,0.000132,0.000131,0.000131,0.000114,0.000000
Z+jets,0.006645,0.000398,0.000398,0.000325,0.000310,0.000309,0.000309,0.000292,0.000000
Total,0.000019,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [8]:
outdir = './csv/Cutflow/Efficiency/'
outname = f'SR_BDT_ctau-{int(ct)}-based_trained_5to100_Background_Efficiency'

df_to_save = bkg_eff_df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/Cutflow/Efficiency//SR_BDT_ctau-10-based_trained_5to100_Background_Efficiency.csv


In [9]:
# weighted event count
pd.options.display.float_format = '{:,.2f}'.format
bkg_cts_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow_cts')

bkg_cts_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
QCD,"38,039.83","1,882.67","1,882.67","1,326.86","1,319.01","1,319.01","1,318.41",632.25,0.00
Top,"5,667.49",559.22,559.22,349.76,314.03,312.99,312.95,294.00,0.49
DY,"14,495.72",671.44,671.44,623.25,623.25,623.25,623.25,455.65,0.00
Diboson,"4,424.20",307.91,307.91,227.24,213.36,211.27,210.33,189.35,0.22
Triboson,53.00,4.32,4.32,2.85,2.51,2.51,2.50,2.09,0.00
W+jets,"227,212.60","17,009.81","17,009.81","13,645.63","12,937.21","12,911.49","12,907.15","11,204.39",24.86
Z+jets,"142,220.15","8,524.55","8,524.55","6,957.67","6,625.48","6,613.11","6,612.30","6,256.76",5.82
Total,"432,113.00","28,959.92","28,959.92","23,133.27","22,034.86","21,993.63","21,986.91","19,034.50",31.38


In [10]:
outdir = './csv/Cutflow/EventCount_Weighted/'
outname = f'SR_BDT_ctau-{int(ct)}-based_trained_5to100_Background_EventCount_Weighted'

df_to_save = bkg_cts_df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/Cutflow/EventCount_Weighted//SR_BDT_ctau-10-based_trained_5to100_Background_EventCount_Weighted.csv


In [11]:
# raw event count
pd.options.display.float_format = '{:,.0f}'.format
bkg_cts_raw_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow_nevts')

bkg_cts_raw_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
QCD,12311,381,381,160,159,159,158,3,0
Top,49818,4688,4688,3058,2348,2337,2336,2006,7
DY,205,13,13,11,11,11,11,9,0
Diboson,18067,1234,1234,915,860,852,848,767,1
Triboson,2257,170,170,116,106,105,104,92,0
W+jets,697657,48425,48425,37015,35472,35415,35377,29090,80
Z+jets,1281459,67427,67427,52627,50986,50934,50903,44480,41
Total,2061774,122338,122338,93902,89942,89813,89737,76447,129


In [12]:
outdir = './csv/Cutflow/EventCount_Raw/'
outname = f'SR_BDT_ctau-{int(ct)}-based_trained_5to100_Background_EventCount_Raw'

df_to_save = bkg_cts_raw_df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/Cutflow/EventCount_Raw//SR_BDT_ctau-10-based_trained_5to100_Background_EventCount_Raw.csv


In [13]:
nBkg = bkg_cts_df.loc['Total'][-1]

print(f'Total number of backgrounds (weighted) after the last SR cut: {nBkg:.2f}')

Total number of backgrounds (weighted) after the last SR cut: 31.38


In [14]:
pd.options.display.float_format = '{:,.2f}'.format

bkg_composition = pd.concat([pd.DataFrame(bkg_cts_df['BDT']).transpose(), pd.DataFrame(bkg_cts_raw_df['BDT']).transpose()], ignore_index=True)

bkg_composition = bkg_composition.rename(index={0: 'Weighted Counts', 1: 'Raw Counts'})
bkg_composition

,QCD,Top,DY,Diboson,Triboson,W+jets,Z+jets,Total
Weighted Counts,0.00,0.49,0.00,0.22,0.00,24.86,5.82,31.38
Raw Counts,0.00,7.00,0.00,1.00,0.00,80.00,41.00,129.00


## Signal

In [15]:
# Signal
sig_histo = util.load(f'{prefix}/signal{suffix}.coffea')[0]

In [16]:
si = utils.get_signal_point_dict(sig_histo)

pd.options.display.float_format = '{:,.1f}'.format
si

,mchi,dmchi,ctau,m1,m2,delta,name
sig_2018_Mchi-88p0_dMchi-16p0_ctau-10,88.0,16.0,10.0,80.0,96.0,0.2,sig_2018_Mchi-88p0_dMchi-16p0_ctau-10
sig_2018_Mchi-84p0_dMchi-8p0_ctau-10,84.0,8.0,10.0,80.0,88.0,0.1,sig_2018_Mchi-84p0_dMchi-8p0_ctau-10
sig_2018_Mchi-94p5_dMchi-9p0_ctau-10,94.5,9.0,10.0,90.0,99.0,0.1,sig_2018_Mchi-94p5_dMchi-9p0_ctau-10
sig_2018_Mchi-99p0_dMchi-18p0_ctau-10,99.0,18.0,10.0,90.0,108.0,0.2,sig_2018_Mchi-99p0_dMchi-18p0_ctau-10
sig_2018_Mchi-73p5_dMchi-7p0_ctau-10,73.5,7.0,10.0,70.0,77.0,0.1,sig_2018_Mchi-73p5_dMchi-7p0_ctau-10
sig_2018_Mchi-77p0_dMchi-14p0_ctau-10,77.0,14.0,10.0,70.0,84.0,0.2,sig_2018_Mchi-77p0_dMchi-14p0_ctau-10
sig_2018_Mchi-60p0_dMchi-20p0_ctau-10,60.0,20.0,10.0,50.0,70.0,0.4,sig_2018_Mchi-60p0_dMchi-20p0_ctau-10
sig_2018_Mchi-63p0_dMchi-6p0_ctau-10,63.0,6.0,10.0,60.0,66.0,0.1,sig_2018_Mchi-63p0_dMchi-6p0_ctau-10
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,5.5,1.0,10.0,5.0,6.0,0.2,sig_2018_Mchi-5p5_dMchi-1p0_ctau-10
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,55.0,10.0,10.0,50.0,60.0,0.2,sig_2018_Mchi-55p0_dMchi-10p0_ctau-10


In [17]:
cutnames = utils.get_signal_list_of_cuts(sig_histo)

cutnames

['Preselections',
 '0 < n(jet) < 3 & n(good vertex) > 0',
 'computing BDT score',
 'No b-tagged jets',
 'Leading jet |eta| < 2.4',
 'Leading jet pT > 80 GeV',
 'dPhi(MET,leading jet) > 1.5',
 'dPhi(MET,all jets) > 0.75',
 'BDT']

### Efficiency

In [18]:
pd.options.display.float_format = '{:,.6f}'.format

In [19]:
eff_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow')

In [20]:
table = pd.DataFrame.from_dict(eff_cutflow, orient='index')
table.columns = cutnames

In [21]:
points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
sig_2018_Mchi-88p0_dMchi-16p0_ctau-10,0.051928,0.009921,0.009921,0.008256,0.008034,0.008034,0.008034,0.007820,0.004657
sig_2018_Mchi-84p0_dMchi-8p0_ctau-10,0.052317,0.008360,0.008360,0.007155,0.006897,0.006897,0.006897,0.006840,0.003005
sig_2018_Mchi-94p5_dMchi-9p0_ctau-10,0.058330,0.009363,0.009363,0.007922,0.007656,0.007656,0.007656,0.007545,0.003549
sig_2018_Mchi-99p0_dMchi-18p0_ctau-10,0.060097,0.011533,0.011533,0.009790,0.009461,0.009457,0.009457,0.009230,0.005057
sig_2018_Mchi-73p5_dMchi-7p0_ctau-10,0.045232,0.006990,0.006990,0.006015,0.005827,0.005827,0.005827,0.005734,0.002637
sig_2018_Mchi-77p0_dMchi-14p0_ctau-10,0.045542,0.008151,0.008151,0.006929,0.006680,0.006676,0.006676,0.006519,0.003486
sig_2018_Mchi-60p0_dMchi-20p0_ctau-10,0.028663,0.005058,0.005058,0.004260,0.004136,0.004136,0.004136,0.003913,0.002273
sig_2018_Mchi-63p0_dMchi-6p0_ctau-10,0.039166,0.005699,0.005699,0.004893,0.004680,0.004676,0.004676,0.004619,0.001910
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,0.010930,0.001187,0.001187,0.000987,0.000935,0.000931,0.000931,0.000876,0.000262
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,0.031125,0.005252,0.005252,0.004562,0.004329,0.004329,0.004329,0.004260,0.002251


In [22]:
outdir = './csv/Cutflow/Efficiency/'
outname = f'SR_BDT_ctau-based_trained_5to100_Signal_ctau-{int(ct)}_samples_Efficiency'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/Efficiency//SR_BDT_ctau-based_trained_5to100_Signal_ctau-10_samples_Efficiency.csv


### Weighted Event count

In [23]:
pd.options.display.float_format = '{:,.2f}'.format

In [24]:
count_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_cts')

In [25]:
table = pd.DataFrame.from_dict(count_cutflow, orient='index')
table.columns = cutnames

In [26]:
points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
#df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
sig_2018_Mchi-88p0_dMchi-16p0_ctau-10,6.15,1.17,1.17,0.98,0.95,0.95,0.95,0.93,0.55
sig_2018_Mchi-84p0_dMchi-8p0_ctau-10,197.37,31.54,31.54,26.99,26.02,26.02,26.02,25.80,11.34
sig_2018_Mchi-94p5_dMchi-9p0_ctau-10,147.91,23.74,23.74,20.09,19.41,19.41,19.41,19.13,9.00
sig_2018_Mchi-99p0_dMchi-18p0_ctau-10,4.78,0.92,0.92,0.78,0.75,0.75,0.75,0.73,0.40
sig_2018_Mchi-73p5_dMchi-7p0_ctau-10,267.87,41.40,41.40,35.62,34.51,34.51,34.51,33.96,15.62
sig_2018_Mchi-77p0_dMchi-14p0_ctau-10,8.45,1.51,1.51,1.29,1.24,1.24,1.24,1.21,0.65
sig_2018_Mchi-60p0_dMchi-20p0_ctau-10,0.55,0.10,0.10,0.08,0.08,0.08,0.08,0.07,0.04
sig_2018_Mchi-63p0_dMchi-6p0_ctau-10,390.29,56.79,56.79,48.76,46.63,46.60,46.60,46.02,19.03
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,211.62,22.98,22.98,19.10,18.10,18.03,18.03,16.97,5.08
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,18.96,3.20,3.20,2.78,2.64,2.64,2.64,2.60,1.37


In [27]:
outdir = './csv/Cutflow/EventCount_Weighted/'
outname = f'SR_BDT_ctau-based_trained_5to100_Signal_ctau-{int(ct)}_samples_EventCount_Weighted'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/EventCount_Weighted//SR_BDT_ctau-based_trained_5to100_Signal_ctau-10_samples_EventCount_Weighted.csv


In [28]:
# without m1 = 30 GeV
m1 = 30.0

points = (si["ctau"] == ct) & (si["m1"] != m1)

df = table.loc[list(si[points].name)]
df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
sig_2018_Mchi-88p0_dMchi-16p0_ctau-10,6.15,1.17,1.17,0.98,0.95,0.95,0.95,0.93,0.55
sig_2018_Mchi-84p0_dMchi-8p0_ctau-10,197.37,31.54,31.54,26.99,26.02,26.02,26.02,25.80,11.34
sig_2018_Mchi-94p5_dMchi-9p0_ctau-10,147.91,23.74,23.74,20.09,19.41,19.41,19.41,19.13,9.00
sig_2018_Mchi-99p0_dMchi-18p0_ctau-10,4.78,0.92,0.92,0.78,0.75,0.75,0.75,0.73,0.40
sig_2018_Mchi-73p5_dMchi-7p0_ctau-10,267.87,41.40,41.40,35.62,34.51,34.51,34.51,33.96,15.62
sig_2018_Mchi-77p0_dMchi-14p0_ctau-10,8.45,1.51,1.51,1.29,1.24,1.24,1.24,1.21,0.65
sig_2018_Mchi-60p0_dMchi-20p0_ctau-10,0.55,0.10,0.10,0.08,0.08,0.08,0.08,0.07,0.04
sig_2018_Mchi-63p0_dMchi-6p0_ctau-10,390.29,56.79,56.79,48.76,46.63,46.60,46.60,46.02,19.03
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,211.62,22.98,22.98,19.10,18.10,18.03,18.03,16.97,5.08
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,18.96,3.20,3.20,2.78,2.64,2.64,2.64,2.60,1.37


### Raw count

In [29]:
pd.options.display.float_format = '{:,.0f}'.format

In [30]:
raw_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_nevts')

In [31]:
table = pd.DataFrame.from_dict(raw_cutflow, orient='index')
table.columns = cutnames

In [32]:
points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
#df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
sig_2018_Mchi-88p0_dMchi-16p0_ctau-10,7516,1436,1436,1195,1163,1163,1163,1132,674
sig_2018_Mchi-84p0_dMchi-8p0_ctau-10,9944,1589,1589,1360,1311,1311,1311,1300,571
sig_2018_Mchi-94p5_dMchi-9p0_ctau-10,32183,5166,5166,4371,4224,4224,4224,4163,1958
sig_2018_Mchi-99p0_dMchi-18p0_ctau-10,31933,6128,6128,5202,5027,5025,5025,4904,2687
sig_2018_Mchi-73p5_dMchi-7p0_ctau-10,10806,1670,1670,1437,1392,1392,1392,1370,630
sig_2018_Mchi-77p0_dMchi-14p0_ctau-10,10438,1868,1868,1588,1531,1530,1530,1494,799
sig_2018_Mchi-60p0_dMchi-20p0_ctau-10,7429,1311,1311,1104,1072,1072,1072,1014,589
sig_2018_Mchi-63p0_dMchi-6p0_ctau-10,11566,1683,1683,1445,1382,1381,1381,1364,564
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,3168,344,344,286,271,270,270,254,76
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,3609,609,609,529,502,502,502,494,261


In [33]:
outdir = './csv/Cutflow/EventCount_Raw/'
outname = f'SR_BDT_ctau-based_trained_5to100_Signal_ctau-{int(ct)}_samples_EventCount_Raw'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/EventCount_Raw//SR_BDT_ctau-based_trained_5to100_Signal_ctau-10_samples_EventCount_Raw.csv


## S/sqrt(B)

In [34]:
nBkg

31.38325412788968

In [35]:
count_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_cts')
table = pd.DataFrame.from_dict(count_cutflow, orient='index')
table.columns = cutnames

In [36]:
pd.options.display.float_format = '{:,.4f}'.format

In [45]:
df = pd.DataFrame(table.loc[:, cutnames[-1]]) # last cut

m1_list = []
delta_list = []
ctau_list = []

for point in df.index.values:
    sig_dict = ptools.signalPoint(point)
    m1_list.append(sig_dict['m1'])
    delta_list.append(sig_dict['delta'])
    ctau_list.append(sig_dict['ctau'])

df['m1'] = m1_list
df['delta'] = delta_list
df['ctau'] = ctau_list

df = df.sort_values(by=['m1']) # sort by m1

In [46]:
# Missing rows
#df[ (df['m1'] == 60) & (df['delta'] == 0.2) ]

df.loc[len(df.index)] = [0, 60, 0.2, 10]

df = df.sort_values(by=['m1']) # sort by m1

In [47]:
df = df.rename(columns={df.columns[0]: 'Signal'})

df['Background'] = nBkg
df

,Signal,m1,delta,ctau,Background
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,93.7532,5.0000,0.1000,10.0000,31.3833
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,5.0792,5.0000,0.2000,10.0000,31.3833
sig_2018_Mchi-11p0_dMchi-2p0_ctau-10,3.4476,10.0000,0.2000,10.0000,31.3833
sig_2018_Mchi-10p5_dMchi-1p0_ctau-10,96.0922,10.0000,0.1000,10.0000,31.3833
sig_2018_Mchi-22p0_dMchi-4p0_ctau-10,4.1660,20.0000,0.2000,10.0000,31.3833
sig_2018_Mchi-21p0_dMchi-2p0_ctau-10,95.3184,20.0000,0.1000,10.0000,31.3833
sig_2018_Mchi-33p0_dMchi-6p0_ctau-10,839.9524,30.0000,0.2000,10.0000,31.3833
sig_2018_Mchi-31p5_dMchi-3p0_ctau-10,"1,182.2941",30.0000,0.1000,10.0000,31.3833
sig_2018_Mchi-44p0_dMchi-8p0_ctau-10,2.8698,40.0000,0.2000,10.0000,31.3833
sig_2018_Mchi-42p0_dMchi-4p0_ctau-10,69.9462,40.0000,0.1000,10.0000,31.3833


In [48]:
df['S/sqrt(B)'] = df['Signal'] / np.sqrt(df['Background'])

df = df[['m1', 'delta', 'ctau', 'Signal', 'Background', 'S/sqrt(B)']]
df

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,5.0000,0.1000,10.0000,93.7532,31.3833,16.7354
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,5.0000,0.2000,10.0000,5.0792,31.3833,0.9067
sig_2018_Mchi-11p0_dMchi-2p0_ctau-10,10.0000,0.2000,10.0000,3.4476,31.3833,0.6154
sig_2018_Mchi-10p5_dMchi-1p0_ctau-10,10.0000,0.1000,10.0000,96.0922,31.3833,17.1530
sig_2018_Mchi-22p0_dMchi-4p0_ctau-10,20.0000,0.2000,10.0000,4.1660,31.3833,0.7437
sig_2018_Mchi-21p0_dMchi-2p0_ctau-10,20.0000,0.1000,10.0000,95.3184,31.3833,17.0148
sig_2018_Mchi-33p0_dMchi-6p0_ctau-10,30.0000,0.2000,10.0000,839.9524,31.3833,149.9359
sig_2018_Mchi-31p5_dMchi-3p0_ctau-10,30.0000,0.1000,10.0000,"1,182.2941",31.3833,211.0457
sig_2018_Mchi-44p0_dMchi-8p0_ctau-10,40.0000,0.2000,10.0000,2.8698,31.3833,0.5123
sig_2018_Mchi-42p0_dMchi-4p0_ctau-10,40.0000,0.1000,10.0000,69.9462,31.3833,12.4858


In [49]:
outdir = './csv/SR_Yields/'
outname = f'SR_BDT_ctau-{int(ct)}-based_trained_5to100_S_over_B'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/SR_Yields//SR_BDT_ctau-10-based_trained_5to100_S_over_B.csv


### ctau = 1mm signal points

In [50]:
pd.options.display.float_format = '{:,.4f}'.format

# S/sqrt(B) in ctau = 1mm, delta = 0.1
delta = 0.1

df_to_show = df[ (df.ctau == ct) & (df.delta == delta) ]
df_to_show.loc[:, 'm1'] = df_to_show.loc[:, 'm1'].map('{:,.1f}'.format)
df_to_show.loc[:, 'delta'] = df_to_show.loc[:, 'delta'].map('{:,.1f}'.format)
df_to_show.loc[:, 'ctau'] = df_to_show.loc[:, 'ctau'].map('{:,.1f}'.format)

df_to_show

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,5.0,0.1,10.0,93.7532,31.3833,16.7354
sig_2018_Mchi-10p5_dMchi-1p0_ctau-10,10.0,0.1,10.0,96.0922,31.3833,17.1530
sig_2018_Mchi-21p0_dMchi-2p0_ctau-10,20.0,0.1,10.0,95.3184,31.3833,17.0148
sig_2018_Mchi-31p5_dMchi-3p0_ctau-10,30.0,0.1,10.0,"1,182.2941",31.3833,211.0457
sig_2018_Mchi-42p0_dMchi-4p0_ctau-10,40.0,0.1,10.0,69.9462,31.3833,12.4858
sig_2018_Mchi-52p5_dMchi-5p0_ctau-10,50.0,0.1,10.0,29.3278,31.3833,5.2352
sig_2018_Mchi-63p0_dMchi-6p0_ctau-10,60.0,0.1,10.0,19.0304,31.3833,3.3970
sig_2018_Mchi-73p5_dMchi-7p0_ctau-10,70.0,0.1,10.0,15.6174,31.3833,2.7878
sig_2018_Mchi-84p0_dMchi-8p0_ctau-10,80.0,0.1,10.0,11.3354,31.3833,2.0234
sig_2018_Mchi-94p5_dMchi-9p0_ctau-10,90.0,0.1,10.0,8.9984,31.3833,1.6063


In [51]:
pd.options.display.float_format = '{:,.4f}'.format

# S/sqrt(B) in ctau = 1mm, delta = 0.2
delta = 0.2

df_to_show = df[ (df.ctau == ct) & (df.delta == delta) ]
df_to_show.loc[:, 'm1'] = df_to_show.loc[:, 'm1'].map('{:,.1f}'.format)
df_to_show.loc[:, 'delta'] = df_to_show.loc[:, 'delta'].map('{:,.1f}'.format)
df_to_show.loc[:, 'ctau'] = df_to_show.loc[:, 'ctau'].map('{:,.1f}'.format)

df_to_show

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,5.0,0.2,10.0,5.0792,31.3833,0.9067
sig_2018_Mchi-11p0_dMchi-2p0_ctau-10,10.0,0.2,10.0,3.4476,31.3833,0.6154
sig_2018_Mchi-22p0_dMchi-4p0_ctau-10,20.0,0.2,10.0,4.1660,31.3833,0.7437
sig_2018_Mchi-33p0_dMchi-6p0_ctau-10,30.0,0.2,10.0,839.9524,31.3833,149.9359
sig_2018_Mchi-44p0_dMchi-8p0_ctau-10,40.0,0.2,10.0,2.8698,31.3833,0.5123
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,50.0,0.2,10.0,1.3715,31.3833,0.2448
22,60.0,0.2,10.0,0.0000,31.3833,0.0000
sig_2018_Mchi-77p0_dMchi-14p0_ctau-10,70.0,0.2,10.0,0.6467,31.3833,0.1154
sig_2018_Mchi-88p0_dMchi-16p0_ctau-10,80.0,0.2,10.0,0.5514,31.3833,0.0984
sig_2018_Mchi-99p0_dMchi-18p0_ctau-10,90.0,0.2,10.0,0.4020,31.3833,0.0718
